In [ ]:
# pip install pandas numpy mplfinance matplotlib ta fyers-apiv3

In [24]:
from fyers_api import fyersModel
from fyers_api import accessToken
from fyers_apiv3 import fyersModel
import pandas as pd
import time
import numpy as np
import math
from datetime import date
import mplfinance as mpf
from ta.trend import MACD
import matplotlib.pyplot as plt
import ta
from urllib.parse import urlparse, parse_qs
import sys

sys.path.append("F:/GitHub/pythonenv/PythonEnvProjects")
from PositionManager import PositionManager

In [28]:
client_id = "3IX55X9A3L-100"
secret_key = "C6XF65EHHK"
redirect_uri = "https://trade.fyers.in/api-login/redirect-uri/index.html"

In [29]:
session = accessToken.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type="code",
    grant_type="authorization_code",
)


response = session.generate_authcode()

In [30]:
print(response)

https://api.fyers.in/api/v1/auth?client_id=3IX55X9A3L-100&redirect_uri=https%3A%2F%2Ftrade.fyers.in%2Fapi-login%2Fredirect-uri%2Findex.html&response_type=code&state=None


In [ ]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MTQ5MzMwMDAsImV4cCI6MTcxNDk2MzAwMCwibmJmIjoxNzE0OTMyNDAwLCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJZSzA3MDk2Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiZjEzMTc5N2VlMjVmN2I4YTRjZjExNTU0OWFlMDM1ODIyMzA5YzczN2FjNDIwNGRiNGVmNGRmNmMiLCJub25jZSI6IiIsImFwcF9pZCI6IlZWQ0ZUQ0I5WTciLCJ1dWlkIjoiZDdhNWU3OGRlN2ZkNGFiYjk3NWE2MzRmOWM0ODc5ZGIiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.ecs6BepmH97Mgsbi2_LrIUWkTa2hO2w-XunGXfwYkJ0"

In [ ]:
session.set_token(auth_code)
response = session.generate_token()
access_token = response["access_token"]
# fyers = fyersModel.FyersModel(
#     client_id=client_id, token=access_token, log_path="C:\\fyersdata"
# )

In [ ]:
# Initialize PositionManager instance
position_manager = PositionManager()

In [ ]:
responseValidate = fyers.get_profile()
print(responseValidate)

In [ ]:
# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=access_token, log_path="C:\\fyersdata"
)

fyersData = fyers.history(data=data)
print(fyersData)

In [ ]:
def get_fyers_data(expiry, symbol, start_date, end_date):
    try:
        # Get historical data
        data = {
            "expiry": expiry,
            "symbol": symbol,
            "resolution": "1",
            "date_format": "1",
            "range_from": start_date,
            "range_to": end_date,
        }
        response = fyers.history(data=data)
        data = response["data"]["candles"]

        # Process data
        df = pd.DataFrame(
            data, columns=["Date", "Open", "High", "Low", "Close", "Volume"]
        )
        df["Date"] = pd.to_datetime(df["Date"], unit="s")
        df.set_index("Date", inplace=True)
        df.sort_index(inplace=True)
        df.index = pd.to_datetime(df.index)
        return df
    except AuthorizationException as e:
        print(e.message)
        return None

In [ ]:
# Function to select the expiration date based on some predefined logic
def select_expiration_date():
    # Example: Choose the nearest Friday as the expiration date
    today = dt.date.today()
    friday = today + dt.timedelta((4 - today.weekday()) % 7)
    return friday.strftime("%Y-%m-%d")

In [ ]:
def get_fyers_data(expiry, symbol, start_date, end_date):
    try:
        # Get historical data
        data = {
            "expiry": expiry,
            "symbol": symbol,
            "resolution": "1",
            "date_format": "1",
            "range_from": start_date,
            "range_to": end_date,
        }
        response = fyers.history(data=data)
        data = response["data"]["candles"]

        # Process data
        df = pd.DataFrame(
            data, columns=["Date", "Open", "High", "Low", "Close", "Volume"]
        )
        df["Date"] = pd.to_datetime(df["Date"], unit="s")
        df.set_index("Date", inplace=True)
        df.sort_index(inplace=True)
        df.index = pd.to_datetime(df.index)
        return df
    except AuthorizationException as e:
        print(e.message)
        return None

In [ ]:
def long_options_strategy(data):
    # Calculate MACD
    data["MACD"], data["Signal"], _ = ta.trend.macd(data["Close"])

    # Generate entry and exit signals
    entry_points_call = []
    entry_points_put = []
    exit_points_call = []

    for i in range(1, len(data)):
        # Entry condition for Long Call
        if data["MACD"][i] > data["Signal"][i] and data["Close"][i] > data["ema_9"][i]:
            entry_points_call.append(data.index[i])

        # Entry condition for Long Put
        if data["MACD"][i] < data["Signal"][i] and data["Close"][i] < data["ema_9"][i]:
            entry_points_put.append(data.index[i])

        # Exit condition for both Long Call and Long Put
        if data["Close"][i] < data["ema_9"][i]:
            exit_points_call.append(data.index[i])

    return entry_points_call, entry_points_put, exit_points_call

In [ ]:
def backtest_strategy(data):
    # Add 9-period exponential moving average (EMA)
    data["ema_9"] = data["Close"].ewm(span=9, adjust=False).mean()

    # Generate trade signals
    entry_points_call, entry_points_put, exit_points_call = long_options_strategy(data)

    # Backtest
    positions = []
    for i in range(len(data)):
        if data.index[i] in entry_points_call:
            position = position_manager.open_position(
                "Call", data["Close"][i], data.index[i], "Long Call"
            )
            position.set_stop_loss(
                data["Low"][i] * 0.95
            )  # Set stop-loss 5% below entry price
            position.set_take_profit(
                data["High"][i] * 1.1
            )  # Set take-profit 10% above entry price
            positions.append(position)
        elif data.index[i] in entry_points_put:
            position = position_manager.open_position(
                "Put", data["Close"][i], data.index[i], "Long Put"
            )
            position.set_stop_loss(
                data["High"][i] * 1.05
            )  # Set stop-loss 5% above entry price
            position.set_take_profit(
                data["Low"][i] * 0.9
            )  # Set take-profit 10% below entry price
            positions.append(position)
        elif data.index[i] in exit_points_call:
            position_manager.close_position(0, data["Close"][i], data.index[i])

    return positions

In [ ]:
def main():
    # Fetch historical data from Fyers API
    expiry = "NSE:NIFTY24509"
    symbol = "NSE:NIFTY50-INDEX"
    start_date = "2023-01-01"
    end_date = "2023-01-31"
    ohlc = get_fyers_data(expiry, symbol, start_date, end_date)

    if ohlc is not None:
        # Backtest strategy
        positions = backtest_strategy(ohlc)

        # Print trade information
        for position in positions:
            print(
                f"Position Type: {position.position_type}, Entry Price: {position.entry_price}, Entry Time: {position.entry_time}"
            )

        # Plot candlestick chart with entry and exit points
        mpf.plot(
            ohlc,
            type="candle",
            volume=False,
            style="yahoo",
            title="Nifty50",
            ylabel="Price",
            addplot=[
                mpf.make_addplot(ohlc["ema_9"]),
                mpf.make_addplot(
                    ohlc.loc[positions.index]["Close"],
                    scatter=True,
                    markersize=100,
                    marker="x",
                    color="r",
                ),
            ],
            figscale=2,
            figsize=(15, 8),
        )
    else:
        print("Failed to fetch data from Fyers API")


if __name__ == "__main__":
    main()